Imports

In [15]:
# Trading imports
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest, GetPortfolioHistoryRequest
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus
# Data imports
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.enums import DataFeed
# Broker imports
from alpaca.broker.client   import BrokerClient


from datetime import datetime
import os
from time import sleep

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.enums import DataFeed

from copulas.multivariate import GaussianMultivariate
from copulas.visualization import scatter_2d
from scipy.stats import gaussian_kde

from scipy.stats import zscore
from scipy.stats import norm



Alpaca connection

In [13]:

# Parsing local API key data
import configparser
config = configparser.ConfigParser()
config.read('local.ini')
API_KEY = None
API_SECRET = None
API_KEY    = config.get('Alpaca-Paper-Trading', 'API_KEY', fallback=API_KEY)
API_SECRET = config.get('Alpaca-Paper-Trading', 'API_SECRET', fallback=API_SECRET)

# This is the API interface to access Alpaca's paper trading account
trading_client = TradingClient(API_KEY, API_SECRET, paper=True) 

data_client = StockHistoricalDataClient(API_KEY, API_SECRET) # Historical data client

# Print account status & cash balance
account = trading_client.get_account()
print(f"Account status: {account.status}")
print(f"Cash balance:    ${account.cash}")



Account status: AccountStatus.ACTIVE
Cash balance:    $100000


Historical Data

In [ ]:
from datetime import datetime

# Set today's date as string
today_str = datetime.today().strftime("%Y-%m-%d")

# Define stock parameters with dynamic end date
stock_params = {
    "META": ("2023-01-01", today_str), # F
    "AAPL": ("2023-01-01", today_str), # A
    "AMZN": ("2023-01-01", today_str), # A
    "NFLX": ("2023-01-01", today_str), # N
    "GOOG": ("2023-01-01", today_str), # G
    "MSFT": ("2023-01-01", today_str),
    "NVDA": ("2023-01-01", today_str),
}

# Loop through and fetch data
for symbol, (start, end) in stock_params.items():
    bars_req = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=TimeFrame.Day,
        start=start,
        end=end,
        feed=DataFeed.IEX
    )
    bars_result = data_client.get_stock_bars(bars_req)
    bars_result.df.to_csv(f"case_data/{symbol}.csv", index=True)


In [ ]:
# Get today's date if not there
today = pd.to_datetime(datetime.today().date())
today_str = today.strftime("%Y-%m-%d")

# Define stock parameters
stock_params = {
    "META": ("2023-01-01", today_str),
    "AAPL": ("2023-01-01", today_str),
    "AMZN": ("2023-01-01", today_str),
    "NFLX": ("2023-01-01", today_str),
    "GOOG": ("2023-01-01", today_str),
    "MSFT": ("2023-01-01", today_str),
    "NVDA": ("2023-01-01", today_str),
}

for symbol, (start, end) in stock_params.items():
    file_path = f"case_data/{symbol}.csv"

    # Load existing data or create empty DataFrame
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
    else:
        df = pd.DataFrame()

    # Check if today is already in data
    if today not in df.index:
        bars_req = StockBarsRequest(
            symbol_or_symbols=[symbol],
            timeframe=TimeFrame.Day,
            start=today_str,
            end=today_str,
            feed=DataFeed.IEX
        )
        bars_result = data_client.get_stock_bars(bars_req)
        new_data = bars_result.df

        if not new_data.empty:
            df = pd.concat([df, new_data])
            df = df[~df.index.duplicated()]  # Remove potential duplicates
            df.to_csv(file_path, index=True)


# Log Returns

# List of tickers
tickers = list(stock_params.keys())

# Dictionary to hold log returns
log_returns = {}

# Load data and compute log returns
for ticker in tickers:
    df = pd.read_csv(f"case_data/{ticker}.csv", index_col=1, parse_dates=True)
    df[f'{ticker}_log'] = np.log(df['close'] / df['close'].shift(1))
    log_returns[f'{ticker}_log'] = df[f'{ticker}_log']

# Combine into single DataFrame
data = pd.DataFrame(log_returns).dropna()

# Remove extreme outliers 
cutoff = 6 # Key step
data_clean = data[(np.abs(zscore(data)) < cutoff).all(axis=1)] 



# GARCH

from arch import arch_model

resids = pd.DataFrame()
aic_results = []

for col in data_clean.columns:
    # Use only the last 252 observations
    scaled_series = 100 * data_clean[col].dropna().iloc[-252:]
    
    model = arch_model(scaled_series, vol='Garch', p=1, q=1, dist='normal')
    fitted = model.fit(disp='off')
    
    resids[col] = fitted.resid / fitted.conditional_volatility


# Fit Copula
copula = GaussianMultivariate()
copula.fit(resids)


# Generate signals

today = pd.to_datetime(datetime.today().date())  # Ensure same format as resids.index
signals_dict = {}

# Transform residuals to uniform marginals
resids_uniform = pd.DataFrame(norm.cdf(resids), columns=resids.columns, index=resids.index)

# Joint likelihoods (for multivariate analysis)
likelihoods = copula.probability_density(resids_uniform)

# Compute thresholds
lower_thresh = np.percentile(likelihoods, 0)
upper_thresh = np.percentile(likelihoods, 30)

# Define extreme days
extreme_days = (likelihoods >= lower_thresh) & (likelihoods < upper_thresh)

for asset in stock_params.keys():
    resid_col = f"{asset}_log"
    
    # Skip if today's not in index
    if today not in resids.index:
        continue
    
    if extreme_days.loc[today]:
        resid_val = resids.at[today, resid_col]
        if resid_val < 0:
            signal = 'buy'
        else:
            signal = 'sell'
    else:
        signal = 'hold'
    
    signals_dict[asset] = signal





# Get META signal
meta_signal = signals_dict.get("META")

positions = {p.symbol: float(p.qty) for p in trading_client.get_all_positions()}
cash = float(account.cash)


# Sell if signal is 'sell' and you hold META
if meta_signal == 'sell' and "META" in positions and positions["META"] > 0:
    qty = int(positions["META"])
    order_req = MarketOrderRequest(
        symbol="META",
        qty=qty,
        side=OrderSide.SELL,
        time_in_force=TimeInForce.DAY
    )
    trading_client.submit_order(order_req)

# Buy if signal is 'buy' and have cash
elif meta_signal == 'buy' and cash > 0:
    df = pd.read_csv("case_data/META.csv", index_col=1, parse_dates=True)
    price = df['close'].iloc[-1]
    qty = int(cash // price)

    if qty > 0:
        order_req = MarketOrderRequest(
            symbol="META",
            qty=qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.DAY
        )
        trading_client.submit_order(order_req)


